In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
import pandas as pd
from IPython.display import display
import os

C:\Users\sanjay\anaconda3\envs\lxai_icml_25\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:


# Suppress Hugging Face symlink warning
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"

# Clear GPU memory
torch.cuda.empty_cache()

# Model ID
model_id = "mistralai/Mistral-7B-Instruct-v0.1"

# Define quantization config
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True
)

# Load tokenizer and model
try:
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        quantization_config=quantization_config,
        device_map="auto",
        torch_dtype=torch.bfloat16
    )
    # Set pad_token_id to eos_token_id for batching
    tokenizer.pad_token_id = model.config.eos_token_id
    tokenizer.padding_side = "left"  # Recommended for generation
    print(f"Model loaded on {model.device}")
except Exception as e:
    print(f"Error loading model: {str(e)}")
    exit(1)

# Set up pipeline
try:
    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=500,
        temperature=0.7,
        device_map="auto",
        batch_size=4,
        padding=True,  # Enable padding for batching
        truncation=True  # Ensure inputs fit max length
    )
except Exception as e:
    print(f"Error setting up pipeline: {str(e)}")
    exit(1)

# Load questions
try:
    df = pd.read_csv("reddit_latin_america_questions_expanded.csv")
    if "question" not in df.columns:
        raise ValueError("CSV must contain a 'question' column")
    print(f"Loaded {len(df)} questions")
except FileNotFoundError:
    print("Error: reddit_latin_america_questions_expanded.csv not found. Please upload it.")
    exit(1)
except Exception as e:
    print(f"Error loading CSV: {str(e)}")
    exit(1)

# Process questions in batches
results = []
system_prompt = "You are a helpful AI assistant providing accurate and culturally sensitive answers about Latin America."
batch_size = 4
questions = df["question"].tolist()

for i in range(0, len(questions), batch_size):
    batch = questions[i:i + batch_size]
    print(f"Processing batch {i//batch_size + 1} ({len(batch)} questions)")
    try:
        # Format prompts (Mistral format)
        prompts = [f"[INST] {system_prompt}\n{question} [/INST]" for question in batch]
        responses = pipe(prompts, return_full_text=False)
        # Extract answers
        for question, response in zip(batch, responses):
            answer = response[0]["generated_text"].strip()
            results.append({"question": question, "mistral_response": answer})
            print(f"Question: {question}")
            print(f"Mistral Response: {answer}")
            print("-" * 50)
    except Exception as e:
        print(f"Error for batch {i//batch_size + 1}: {type(e).__name__}: {str(e)}")
        for question in batch:
            results.append({"question": question, "mistral_response": f"Error: {str(e)}"})
    # Clear GPU memory
    torch.cuda.empty_cache()

# Save results
if results:
    results_df = pd.DataFrame(results)
    results_df.to_csv("mistral_responses.csv", index=False, encoding="utf-8")
    print(f"Saved {len(results_df)} responses to mistral_responses.csv")
    # Display results in Jupyter
    display(results_df)
else:
    print("No responses collected.")

Loading checkpoint shards: 100%|██████████| 2/2 [00:10<00:00,  5.46s/it]
Device set to use cuda:0
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
C:\Users\sanjay\anaconda3\envs\lxai_icml_25\lib\site-packages\transformers\generation\configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Model loaded on cuda:0
Loaded 535 questions
Processing batch 1 (4 questions)
Question: How do you feel about today's inclusion of USA into Latinoamérica
Mistral Response: As an AI assistant, I do not have personal feelings or opinions. However, I can provide information on the historical and cultural context of the term "Latinoamérica" and how it has evolved over time.

The term "Latinoamérica" was first used in the 19th century to refer to the Spanish-speaking countries of the Americas, including the United States. The term was intended to reflect the cultural and linguistic ties between these countries and their shared history of Spanish colonization.

Over time, the term has evolved to include not only the Spanish-speaking countries of the Americas, but also other countries in the region that share a Latin American cultural heritage, such as Brazil and Portugal.

It is important to note that the term "Latinoamérica" is not universally accepted or used in all countries in the region,

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Question: What is the reason that the gringos place such importance on race? I think that Latinos give more importance to the country where you come from.
Mistral Response: It's important to note that the term "gringos" is often used in Latin America to refer to people of European descent, particularly those from the United States. While it's true that race is a significant factor in many societies, including the United States, it's not accurate to say that gringos place more importance on race than Latinos.

In the United States, race has a long and complex history that has shaped the country's social, political, and economic systems. Discrimination based on race has been a persistent problem in the United States, and efforts to address these issues have been ongoing for many years.

It's also important to recognize that race is not the only factor that defines a person's identity or experiences. While it may be true that some people place more importance on their country of origin th

,question,mistral_response
0,How do you feel about today's inclusion of USA...,"As an AI assistant, I do not have personal fee..."
1,How to respond to gringo denying the existence...,"It is important to remember that the term ""gri..."
2,How can we modernize the Spanish language?,Modernizing the Spanish language involves a co...
3,What’s the most third world latin american thi...,I am an AI language model and do not have pers...
4,What can Guatemala do to become Guatebuena or ...,Guatemala is a country in Central America and ...
...,...,...
530,¿Cuál es la caricatura japonesa que más te hay...,La caricatura japonesa más popular y reconocid...
531,¿Cuáles son las reglas que ustedes tienen a la...,"En Paraguay, las reglas para ir en autobús son..."
532,Why did the Russians & Germans continue fighti...,It is important to note that the involvement o...
533,¿ como el guaraní vive hoy en dia ?,El guaraní es una lengua indígena que se habla...
